In [1]:
# conv, group conv, separable conv, depth wise conv, point wise conv
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, SeparableConv2D, DepthwiseConv2D, BatchNormalization, ReLU, GlobalAveragePooling2D, Flatten, Dense, MaxPool2D
from tensorflow.keras import Sequential, Model
import tensorflow_datasets as tfds
import tqdm, os, time

2024-04-12 16:48:46.681021: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 16:48:46.681048: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 16:48:46.681476: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 16:48:46.727322: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 16:48:47.504584: W tensorflow/compiler/

In [2]:
tf.__version__

'2.14.0'

In [3]:
ic, oc = 20, 30
g = 10
input = tf.zeros([1, 20, 20, ic])
conv = Conv2D(oc, [3,3], padding='same') # normal
g_conv = Conv2D(oc, [3,3], groups=g, padding='same') # group
s_conv = SeparableConv2D(oc, (3,3), padding='same') # depth wise & point wise
d_conv = DepthwiseConv2D((3,3), padding='same') # depth wise
p_conv = Conv2D(oc, [1,1]) # point wise

2024-04-12 16:48:48.506852: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-12 16:48:48.584931: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-12 16:48:48.585115: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
model = Sequential([conv])
model(input).shape, model.summary(), 3**2*ic*oc+oc

2024-04-12 16:48:50.536844: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (1, 20, 20, 30)           5430      
                                                                 
Total params: 5430 (21.21 KB)
Trainable params: 5430 (21.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(TensorShape([1, 20, 20, 30]), None, 5430)

In [5]:
model = Sequential([g_conv])
model(input).shape, model.summary(), (3**2*ic*oc)//g+oc

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (1, 20, 20, 30)           570       
                                                                 
Total params: 570 (2.23 KB)
Trainable params: 570 (2.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


2024-04-12 16:48:51.368140: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5c807aabe480 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-04-12 16:48:51.368169: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-04-12 16:48:51.535398: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


(TensorShape([1, 20, 20, 30]), None, 570)

In [6]:
model = Sequential([s_conv])
model(input).shape, model.summary(), 3**2*ic + ic*oc+oc

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 separable_conv2d (Separabl  (1, 20, 20, 30)           810       
 eConv2D)                                                        
                                                                 
Total params: 810 (3.16 KB)
Trainable params: 810 (3.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(TensorShape([1, 20, 20, 30]), None, 810)

In [7]:
model = Sequential([d_conv])
model(input).shape, model.summary(), 3**2*ic+ic

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 depthwise_conv2d (Depthwis  (1, 20, 20, 20)           200       
 eConv2D)                                                        
                                                                 
Total params: 200 (800.00 Byte)
Trainable params: 200 (800.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(TensorShape([1, 20, 20, 20]), None, 200)

In [8]:
model = Sequential([d_conv, p_conv])
model(input).shape, model.summary(), 3**2*ic+ic + ic*oc+oc

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 depthwise_conv2d (Depthwis  (1, 20, 20, 20)           200       
 eConv2D)                                                        
                                                                 
 conv2d_2 (Conv2D)           (1, 20, 20, 30)           630       
                                                                 
Total params: 830 (3.24 KB)
Trainable params: 830 (3.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


(TensorShape([1, 20, 20, 30]), None, 830)

In [9]:
weights = tf.keras.initializers.GlorotUniform(seed=42)
bias = tf.keras.initializers.GlorotUniform(seed=42)

class CBA(Layer):
    def __init__(self, conv, oc, kernel, groups=1, strides=(1,1), padding='same', bn=True, act=True):
        super().__init__()
        if conv == SeparableConv2D:
            self.conv = conv(oc, kernel, strides=strides, padding=padding, kernel_initializer=weights, bias_initializer=bias)
        else:
            self.conv = conv(oc, kernel, groups=groups, strides=strides, padding=padding, kernel_initializer=weights, bias_initializer=bias)
        self.bn = BatchNormalization() if bn else Identity()
        self.act = ReLU() if act else Identity()

    def call(self, x, training=False):
        x = self.conv(x)
        x = self.bn(x, training)
        x = self.act(x)
        return x


class Identity(Layer):
    def __init__(self):
        super().__init__()

    def call(self, x, training=False):
        return x

    def train(self, x, arg=False):
        return x


class ResBlock(Layer):
    def __init__(self, conv, oc, kernel, groups=1, strides=1, padding='same'):
        super().__init__()
        self.conv1 = CBA(conv, oc, kernel, groups=groups, strides=strides, padding=padding)
        self.conv2 = CBA(conv, oc, kernel, groups=groups, strides=1, padding=padding, act=False)
        self.res_conv = CBA(conv, oc, 1, groups=groups, strides=strides, padding=padding, act=False) if strides != 1 else Identity()

    def call(self, x, training=False):
        residual = self.res_conv(x, training)
        x = self.conv1(x, training)
        x = self.conv2(x, training)
        return x + residual


class ResNet18(Model):
    def __init__(self, conv, g, oc, nc):
        super().__init__()
        if conv == Conv2D:
            if g == 1:
                self.conv_name = 'Conv'
            else:
                self.conv_name = 'GroupConv'
        elif conv == SeparableConv2D:
            self.conv_name = 'SeparableConv'
        self.oc = oc

        self.conv = CBA(conv, oc, 7, g, (2,2), 'same')
        self.maxpool = MaxPool2D((3,3), strides=(2,2), padding='same')
        g = oc if g != 1 else 1

        self.res1 = [ResBlock(conv, oc, 3, g, 1, 'same'),
                     ResBlock(conv, oc, 3, g, 1, 'same')]
        g = oc if g != 1 else 1

        self.res2 = [ResBlock(conv, oc*2, 3, g, 2, 'same'),
                     ResBlock(conv, oc*2, 3, g, 1, 'same')]
        g = oc*2 if g != 1 else 1


        self.res3 = [ResBlock(conv, oc*4, 3, g, 2, 'same'),
                     ResBlock(conv, oc*4, 3, g, 1, 'same')]
        g = oc*4 if g != 1 else 1

        self.res4 = [ResBlock(conv, oc*8, 3, g, 2, 'same'),
                     ResBlock(conv, oc*8, 3, g, 1, 'same')]

        self.gap = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.linear = Dense(nc, kernel_initializer=weights, bias_initializer=bias)

    def call(self, x, training=False):
        x = self.conv(x, training)
        x = self.maxpool(x)
        for res in self.res1:
            x = res(x, training)
        for res in self.res2:
            x = res(x, training)
        for res in self.res3:
            x = res(x, training)
        for res in self.res4:
            x = res(x, training)
        x = self.gap(x)
        x = self.flatten(x)
        x = self.linear(x)
        return x

In [10]:
def resize_padding(image, random=False, mode='CONSTANT', constant_values=0, seed=42):
    size = tf.cast(tf.shape(image)[:2][::-1], tf.float32)
    ratio = img_size/tf.reduce_max(size)
    new_size = tf.cast(ratio * size, tf.int32)
    pad_size = img_size - tf.cast(new_size, tf.float32)

    pad_ratio = tf.random.uniform((), 0, 1, seed=seed) if random else 0.5
    pad_LT = tf.cast(pad_size*pad_ratio, tf.int32)
    pad_RB = tf.cast(pad_size, tf.int32) - pad_LT
    pad_left, pad_top = tf.unstack(pad_LT)
    pad_right, pad_bottom = tf.unstack(pad_RB)
    padding = tf.reshape(tf.stack([pad_top, pad_bottom, pad_left, pad_right, 0, 0]), [3, 2])

    resized_image = tf.image.resize(image, new_size[::-1])
    padded_image = tf.pad(resized_image, padding, mode=mode, constant_values=constant_values)

    return padded_image

def preprocessing(name, image, label):
    if name == 'mnist':
        image = tf.image.grayscale_to_rgb(image)
    image = resize_padding(image)
    return tf.cast(image, tf.float32) / 255., label

In [11]:
tf.random.set_seed(42)
tf.keras.utils.set_random_seed(42)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

def get_data(name):
    data, info = tfds.load(name, with_info=True, as_supervised=True, data_dir='./data')
    train_data = data['train']
    train_data = train_data.map(lambda x, y: preprocessing(name, x, y), num_parallel_calls=tf.data.AUTOTUNE)
    train_data = train_data.batch(batch_size, drop_remainder=True, num_parallel_calls=-1).cache().shuffle(60000).prefetch(tf.data.AUTOTUNE)
    test_data = data['test']
    test_data = test_data.map(lambda x, y: preprocessing(name, x, y), num_parallel_calls=tf.data.AUTOTUNE)
    test_data = test_data.batch(batch_size, drop_remainder=True, num_parallel_calls=-1).cache().prefetch(tf.data.AUTOTUNE)

    return train_data, test_data

def get_opt_loss(lr):
    optimizer = tf.keras.optimizers.SGD(learning_rate=lr)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    return optimizer, loss

In [12]:
def train_test(data, model):
    def train():
        optimizer, loss = get_opt_loss(lr)
        @tf.function
        def train_step(x, y):
            with tf.GradientTape() as train_tape:
                preds = model(x, True)
                loss_ = loss(y, preds)
                gradients = train_tape.gradient(loss_, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return loss_

        for epoch in range(epochs):
            train_tqdm = tqdm.tqdm(train_data, total=len(train_data), desc=f'train {model.conv_name} epoch {epoch+1}/{epochs}', ascii=' =', colour='red')
            total_loss = 0
            if epoch < 400:
                optimizer.lr.assign(lr)
            elif epoch < 70:
                optimizer.lr.assign(lr*1e-1)
            elif epoch < 90:
                optimizer.lr.assign(lr*1e-2)
            else:
                optimizer.lr.assign(lr*1e-3)
            for iter, (images, labels) in enumerate(train_tqdm):
                loss_ = train_step(images, labels)
                total_loss += loss_
                loss_ = total_loss / (iter+1)
                train_tqdm.set_postfix_str(f'lr={optimizer.lr.numpy():.4f}, loss={loss_.numpy():.5f}')

    def test():
        correct = 0
        pred_time = 0

        @tf.function
        def test_step(x):
            preds = model(x)
            return preds

        test_tqdm = tqdm.tqdm(test_data, total=len(test_data), desc=f'test {model.conv_name} ', ascii=' =', colour='blue')
        for iter, (images, labels) in enumerate(test_tqdm):
            st = tf.timestamp()
            preds = test_step(images)
            et = tf.timestamp()
            pred_time += (et - st)

            correct += tf.reduce_sum(tf.cast(labels==tf.argmax(preds, -1), tf.int32))
            test_tqdm.set_postfix_str(f'correct: {correct}/{(iter+1)*labels.shape[0]}, time: {pred_time/(iter+1):.5f}')

    train_data, test_data = get_data(data)
    train()
    test()

In [13]:
data = 'mnist'
img_size = 64
epochs = 20
batch_size = 2000
lr=1e-1

oc, nc, ic = 3, 10, 3
g = 1
input = tf.zeros([1, 64, 64, ic])
conv = Conv2D
s_conv = SeparableConv2D

model = ResNet18(conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 10)
Model: "res_net18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba (CBA)                   multiple                  456       
                                                                 
 max_pooling2d (MaxPooling2  multiple                  0         
 D)                                                              
                                                                 
 res_block (ResBlock)        multiple                  192       
                                                                 
 res_block_1 (ResBlock)      multiple                  192       
                                                                 
 res_block_2 (ResBlock)      multiple                  594       
                                                                 
 res_block_3 (ResBlock)      multiple                  708       
                                                 

test Conv : 100%|==========| 5/5 [00:00<00:00,  7.26it/s, correct: 9800/10000, time: 0.10573]


In [14]:
model = ResNet18(s_conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 10)
Model: "res_net18_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba_20 (CBA)                multiple                  171       
                                                                 
 max_pooling2d_1 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 res_block_8 (ResBlock)      multiple                  102       
                                                                 
 res_block_9 (ResBlock)      multiple                  102       
                                                                 
 res_block_10 (ResBlock)     multiple                  246       
                                                                 
 res_block_11 (ResBlock)     multiple                  240       
                                               

test SeparableConv : 100%|==========| 5/5 [00:00<00:00, 11.51it/s, correct: 9667/10000, time: 0.06144]


In [15]:
g=3
model = ResNet18(conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 10)
Model: "res_net18_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba_40 (CBA)                multiple                  162       
                                                                 
 max_pooling2d_2 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 res_block_16 (ResBlock)     multiple                  84        
                                                                 
 res_block_17 (ResBlock)     multiple                  84        
                                                                 
 res_block_18 (ResBlock)     multiple                  258       
                                                                 
 res_block_19 (ResBlock)     multiple                  276       
                                               

test GroupConv : 100%|==========| 5/5 [00:00<00:00,  6.11it/s, correct: 9632/10000, time: 0.13944]


In [16]:
data = 'cifar100'
img_size = 256
epochs = 30
batch_size = 200
lr=1e-1

oc, nc, ic = 60, 100, 3
g = 1
input = tf.zeros([1, 64, 64, ic])
conv = Conv2D
s_conv = SeparableConv2D

model = ResNet18(conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 100)
Model: "res_net18_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba_60 (CBA)                multiple                  9120      
                                                                 
 max_pooling2d_3 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 res_block_24 (ResBlock)     multiple                  65400     
                                                                 
 res_block_25 (ResBlock)     multiple                  65400     
                                                                 
 res_block_26 (ResBlock)     multiple                  203400    
                                                                 
 res_block_27 (ResBlock)     multiple                  260400    
                                              

test Conv : 100%|==========| 50/50 [00:07<00:00,  6.38it/s, correct: 5910/10000, time: 0.06844]


In [17]:
model = ResNet18(s_conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 100)
Model: "res_net18_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba_80 (CBA)                multiple                  627       
                                                                 
 max_pooling2d_4 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 res_block_32 (ResBlock)     multiple                  8880      
                                                                 
 res_block_33 (ResBlock)     multiple                  8880      
                                                                 
 res_block_34 (ResBlock)     multiple                  32280     
                                                                 
 res_block_35 (ResBlock)     multiple                  32160     
                                              

test SeparableConv : 100%|==========| 50/50 [00:04<00:00, 11.23it/s, correct: 5417/10000, time: 0.04423]


In [18]:
g=3
model = ResNet18(conv, g, oc, nc)
print(model(input).shape)
model.summary()
train_test(data, model)

(1, 100)
Model: "res_net18_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cba_100 (CBA)               multiple                  3240      
                                                                 
 max_pooling2d_5 (MaxPoolin  multiple                  0         
 g2D)                                                            
                                                                 
 res_block_40 (ResBlock)     multiple                  1680      
                                                                 
 res_block_41 (ResBlock)     multiple                  1680      
                                                                 
 res_block_42 (ResBlock)     multiple                  5160      
                                                                 
 res_block_43 (ResBlock)     multiple                  5520      
                                              

test GroupConv : 100%|==========| 50/50 [00:05<00:00,  8.79it/s, correct: 3992/10000, time: 0.05554]
